# 1. 접근토큰 발급

In [26]:
# 접근토큰 발급

import yaml
import requests
import json

with open('config.yaml', 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

api_key = config['hantu']['api_key']
secret_key = config['hantu']['secret_key']

In [27]:
base_url = 'https://openapivts.koreainvestment.com:29443'

headers = {"content-type":"application/json"}
body = {
        "grant_type":"client_credentials",
        "appkey":api_key, 
        "appsecret":secret_key,
        }
url = base_url + '/oauth2/tokenP'
res = requests.post(url, headers=headers, data=json.dumps(body)).json()

access_token = res['access_token']

---
# 2. 내 계좌 정보 가져오기

- 현금 잔고 조회
- 보유 종목 조회

## (1) 현금 잔고 조회

- 계좌 조회기 때문에, 내 계좌번호 정보를 전달해야 함. account_id 이름으로 등록.
- account_suffix는 XXXXXXXX-01 이렇게, - 다음에 오는 두 자리 숫자. 첫 계좌라면 보통 01

In [28]:
# 내 계좌 정보 준비

account_id = config['hantu']['account_id']
account_id

account_suffix = '01'

In [29]:
# 현금 잔고 조회

tr_id = 'VTTC8434R'

headers = {"content-type":"application/json",
        "appkey":api_key, 
        "appsecret":secret_key,
        "authorization":f"Bearer {access_token}",
        "tr_id":tr_id,
        }

params = {
"CANO":account_id,
"ACNT_PRDT_CD": account_suffix,
"AFHR_FLPR_YN": "N",
"OFL_YN": "N",
"INQR_DVSN": "01",
"UNPR_DVSN": "01",
"FUND_STTL_ICLD_YN": "N",
"FNCG_AMT_AUTO_RDPT_YN": "N",
"PRCS_DVSN": "01",
"CTX_AREA_FK100": '',
"CTX_AREA_NK100": ''
}

url = base_url + '/uapi/domestic-stock/v1/trading/inquire-balance'

order_result = requests.get(url, headers=headers, params=params).json()


- order_result에서 내 잔고 조회, 보유 종목 조회 정보가 모두 들어있음

In [30]:
# output2에는 현금/평가금액 관련 정보
order_result = order_result['output2'][0]
order_result

{'dnca_tot_amt': '6353991',
 'nxdy_excc_amt': '6353991',
 'prvs_rcdl_excc_amt': '9993545',
 'cma_evlu_amt': '0',
 'bfdy_buy_amt': '0',
 'thdt_buy_amt': '0',
 'nxdy_auto_rdpt_amt': '0',
 'bfdy_sll_amt': '0',
 'thdt_sll_amt': '3645500',
 'd2_auto_rdpt_amt': '0',
 'bfdy_tlex_amt': '0',
 'thdt_tlex_amt': '5946',
 'tot_loan_amt': '0',
 'scts_evlu_amt': '0',
 'tot_evlu_amt': '9993545',
 'nass_amt': '9993545',
 'fncg_gld_auto_rdpt_yn': '',
 'pchs_amt_smtl_amt': '0',
 'evlu_amt_smtl_amt': '0',
 'evlu_pfls_smtl_amt': '0',
 'tot_stln_slng_chgs': '0',
 'bfdy_tot_asst_evlu_amt': '9994591',
 'asst_icdc_amt': '-1046',
 'asst_icdc_erng_rt': '-0.01046566'}

In [31]:
# 보유 현금 조회
float(order_result['prvs_rcdl_excc_amt'])

9993545.0

## (2) 보유 종목 리턴

- 보유 종목이 50개 이상일 경우, 한번에 전체 종목이 조회되지 않음(최대 50종목까지)
- 때문에, while 문으로 반복하며 여러번 조회해야 함

In [32]:
# 보유 종목 불러오기

tr_id = 'VTTC8434R'

headers = {"content-type":"application/json",
        "appkey":api_key, 
        "appsecret":secret_key,
        "authorization":f"Bearer {access_token}",
        "tr_id":tr_id,
        }
output1_result = []
cont = True
ctx_area_fk100 = ''
ctx_area_nk100 = ''

# while문을 돌며 cont = False가 될때까지 반복
while cont:
    params = {
        "CANO":account_id,
        "ACNT_PRDT_CD": account_suffix,
        "AFHR_FLPR_YN": "N",
        "OFL_YN": "N",
        "INQR_DVSN": "01",
        "UNPR_DVSN": "01",
        "FUND_STTL_ICLD_YN": "N",
        "FNCG_AMT_AUTO_RDPT_YN": "N",
        "PRCS_DVSN": "01",
        "CTX_AREA_FK100": ctx_area_fk100,
        "CTX_AREA_NK100": ctx_area_nk100
    }

    url = base_url + '/uapi/domestic-stock/v1/trading/inquire-balance'

    order_result = requests.get(url, headers=headers, params=params)

    received_headers = order_result.headers
    contents = order_result.json()

    # 다음 종목 조회를 위한 전달값 준비. 한투 서버는 이 값을 받아 사용자가 어디까지 조회했는지를 알 수 있다.
    # F, M 값이라면 아직 조회하지 않은 데이터가 있다는 의미
    cont = received_headers['tr_cont'] in ['F','M']

    # 다음 데이터를 조회하려면 'N' 값으로 둬야함
    headers['tr_cont'] = 'N'
    ctx_area_fk100 = contents['ctx_area_fk100']
    ctx_area_nk100 = contents['ctx_area_nk100']

    # 조회된 데이터 저장
    output1_result = output1_result + contents['output1']


In [33]:
# output 값 확인

import pandas as pd

pd.DataFrame(output1_result)

""


---
# 3. 매수/매도 주문

- 지정가/시장가 주문

In [45]:
# 매매 파라미터 설정


ord_dvsn = '00' # 00은 지정가, 01은 시장가
ticker = '005930'
quantity = 1 # 주문수량
price = 80000 # 주문가격

In [51]:
# 매수주문 구현

tr_id = 'TTTC0802U' # TTTC0801U로 바꾸면 매도주문이 됩니다

headers = {"content-type":"application/json",
        "appkey":api_key, 
        "appsecret":secret_key,
        "authorization":f"Bearer {access_token}",
        "tr_id":tr_id,
        }

params = {
        "CANO":account_id,
        "ACNT_PRDT_CD": account_suffix,
        'PDNO':ticker,
        'ORD_DVSN':ord_dvsn,
        'ORD_QTY':str(quantity), # 주문수량/가격은 string 값으로 바꿔서 넣어줍니다
        'ORD_UNPR':str(price)
        }

url = base_url + '/uapi/domestic-stock/v1/trading/order-cash'

order_result = requests.post(url, headers=headers, data=json.dumps(params)) # post 방식은 parameter 입력방식이 다름


- 주문이 제대로 완료됐다면, msg1에 주문 전송 완료 되었습니다. 라는 메세지를 볼 수 있음. rt_cd = 0임으로도 확인 가능
- 만약 주문이 제대로 들어가지 않았다면, rt_cd는 0이 아니고, msg1에 오류 이유가 리턴됨
- output의 ODNO는 내 주문의 주문번호로, 뒤에 배울 주문 취소 기능에 필요한 데이터. 저장해두자.

In [52]:
# 주문 결과 확인

order_result.json()

{'rt_cd': '0',
 'msg_cd': 'APBK0013',
 'msg1': '주문 전송 완료 되었습니다.',
 'output': {'KRX_FWDG_ORD_ORGNO': '91259',
  'ODNO': '0000107940',
  'ORD_TMD': '131420'}}

- 주문이 제대로 들어갔다면, 내 계좌 주문내역에서 아래와 같이 주문내역을 확인할 수 있습니다


![주문내역](6-3주문내역.jpg)

---
# 4. 주문 체결 확인하기

- 내가 넣은 주문이 체결됐는지 확인
- 확인 방법은 크게 3가지
    1. 체결리스트에 내 주문이 생겼는지 확인
    2. 미체결리스트에 내 주문이 사라졌는지 확인
    3. 맨 처음 배운 내 계좌 정보 가져오기 방법으로 잔고가 생겼는지 확인

In [75]:
# 주식 일별 주문 체결 조회 기능 구현

from datetime import datetime
import time

sll_buy_dvsn_cd = '02' # 매도는 01, 매수는 02
ccld_dvsn = '01' # 체결리스트를 받아오려면 01, 미체결은 02
ticker = ''
date = ''.join(str(datetime.now()).split(' ')[0].split('-'))

tr_id = 'TTTC8001R' # TTTC8001R 은 매수, TTTC0801U 은 매도주문

headers = {"content-type":"application/json",
        "appkey":api_key, 
        "appsecret":secret_key,
        "authorization":f"Bearer {access_token}",
        "tr_id":tr_id,
        }

output1_result = []
cont = True
ctx_area_fk100 = ''
ctx_area_nk100 = ''
while cont:
    params = {
        "CANO":account_id,
        "ACNT_PRDT_CD": account_suffix,
        'INQR_STRT_DT':date,
        'INQR_END_DT':date,
        'SLL_BUY_DVSN_CD':sll_buy_dvsn_cd,
        'INQR_DVSN':'00',
        'PDNO':ticker,
        'CCLD_DVSN':ccld_dvsn,
        "ORD_GNO_BRNO": "",
        "ODNO": "",
        "INQR_DVSN_3": "",
        "INQR_DVSN_1": "",
        "CTX_AREA_FK100": ctx_area_fk100,
        "CTX_AREA_NK100": ctx_area_nk100
    }

    url = base_url + '/uapi/domestic-stock/v1/trading/inquire-daily-ccld'
    order_result = requests.post(url, headers=headers, params=params)

    received_headers = order_result.headers
    contents = order_result.json()

    cont = received_headers['tr_cont'] in ['F','M']
    received_headers['tr_cont'] = 'N' # 다음 데이터를 조회하려면 'N' 값으로 둬야함
    ctx_area_fk100 = contents['ctx_area_fk100']
    ctx_area_nk100 = contents['ctx_area_nk100']

    output1_result = output1_result + contents['output1']
            

In [76]:
pd.DataFrame(output1_result)

,ord_dt,ord_gno_brno,odno,orgn_odno,ord_dvsn_name,sll_buy_dvsn_cd,sll_buy_dvsn_cd_name,pdno,prdt_name,ord_qty,...,inqr_ip_addr,cpbc_ordp_ord_rcit_dvsn_cd,cpbc_ordp_infm_mthd_dvsn_cd,infm_tmd,ctac_tlno,prdt_type_cd,excg_dvsn_cd,cpbc_ordp_mtrl_dvsn_cd,ord_orgno,rsvn_ord_end_dt
0,20240719,91259,0000107940,,지정가,02,현금매수,005930,삼성전자,1,...,049.247.006.122,,,,01091187680,300,02,11,00000,


---
# 5. 주문 취소

In [77]:
# 주문 취소 기능 구현

oid = '0000107940' # 앞서 넣은 주문의 ODNO를 기록해두고, 이 값을 활용해 주문 취소 가능

tr_id = 'TTTC0803U'
headers = {"content-type":"application/json",
        "appkey":api_key, 
        "appsecret":secret_key,
        "authorization":f"Bearer {access_token}",
        "tr_id":tr_id,
        }

params = {
        "CANO":account_id,
        "ACNT_PRDT_CD": account_suffix,
        "KRX_FWDG_ORD_ORGNO":"",
        "RVSE_CNCL_DVSN_CD":'02', # 취소
        "ORGN_ODNO":oid, # 원주문번호
        "ORD_DVSN":'00',
        "ORD_QTY":'0',
        "ORD_UNPR":'0',
        "QTY_ALL_ORD_YN":"Y" # 전량취소
        }
url = base_url + '/uapi/domestic-stock/v1/trading/order-rvsecncl'
order_result = requests.post(url, headers=headers, data=json.dumps(params)) # post 방식은 parameter 입력방식이 다름

- 제대로 취소됐다면 rt_cd = 0, msg1은 주문 전송 완료 되었습니다. 라는 메세지가 리턴됨

In [79]:
order_result.json()

{'rt_cd': '0',
 'msg_cd': 'APBK0013',
 'msg1': '주문 전송 완료 되었습니다.',
 'output': {'KRX_FWDG_ORD_ORGNO': '91259',
  'ODNO': '0000118779',
  'ORD_TMD': '140415'}}